In [4]:
import json
import numpy as np
import os
from scipy.linalg import expm, logm
import sys

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp

In [5]:
qc_0 = QuantumCircuit(2)
qc_0.h(0)
qc_0.cx(0, 1)

In [6]:
U_0 = Operator(qc_0)
pauli_op = SparsePauliOp.from_operator(U_0)
pauli_strings = pauli_op.paulis
pauli_labels = [p.to_label() for p in pauli_strings]
print(pauli_labels)
pauli_coeffs = pauli_op.coeffs
print(pauli_coeffs)
U_1 = pauli_op.to_matrix()

['II', 'IX', 'IY', 'IZ', 'XI', 'XX', 'XY', 'XZ']
[ 0.35355339+0.j          0.35355339+0.j          0.        +0.35355339j
  0.35355339+0.j         -0.35355339+0.j          0.35355339+0.j
 -0.        -0.35355339j  0.35355339+0.j        ]


In [7]:
np.allclose(U_0.data, U_1)

True

In [8]:
def maximal_munch(pauli_str):
    chunks = []
    start = None

    for i, c in enumerate(pauli_str):
        if c in {'X', 'Y', 'Z'}:
            if start is None:
                start = i
        else:
            if start is not None:
                chunks.append((start, pauli_str[start:i]))
                start = None

    if start is not None:
        chunks.append((start, pauli_str[start:]))

    return chunks

In [9]:
# --- Basis rotation helpers ---
def apply_basis_change(qc, pauli_str, inverse=False):
    for i, p in enumerate(pauli_str):
        if p == 'I':
            continue
        if inverse:
            if p == 'X':
                qc.h(i)
            elif p == 'Y':
                qc.s(i)
                qc.h(i)
        else:
            if p == 'X':
                qc.h(i)
            elif p == 'Y':
                qc.h(i)
                qc.sdg(i)

# --- Exponential of Pauli string ---
def apply_pauli_exp(qc, pauli_str, theta):
    if abs(theta) < 1e-10:
        return
    qubits = [i for i, p in enumerate(pauli_str) if p != 'I']
    if not qubits:
        return
    apply_basis_change(qc, pauli_str)
    for i in range(len(qubits) - 1):
        qc.cx(qubits[i], qubits[i + 1])
    qc.rz(2 * theta, qubits[-1])
    for i in reversed(range(len(qubits) - 1)):
        qc.cx(qubits[i], qubits[i + 1])
    apply_basis_change(qc, pauli_str, inverse=True)

# --- Build full circuit from PauliOp decomposition ---
def build_pauli_evolution_circuit(pauli_labels, pauli_coeffs, num_qubits):
    qc = QuantumCircuit(num_qubits)
    for label, coeff in zip(pauli_labels, pauli_coeffs):
        if abs(coeff) < 1e-10:
            continue
        if abs(coeff.imag) > 1e-10:
            print(f"⚠️ Complex coeff {coeff} for {label} — using only real part.")
        apply_pauli_exp(qc, label, coeff.real)
    return qc

# --- Reconstruct the circuit ---
qc_exp = build_pauli_evolution_circuit(pauli_labels, pauli_coeffs, 2)
U_exp = Operator(qc_exp)

# --- Compare to original unitary ---
print("Unitary match:", np.allclose(U_0.data, U_exp.data))


⚠️ Complex coeff 0.35355339059327373j for IY — using only real part.
⚠️ Complex coeff (-0-0.35355339059327373j) for XY — using only real part.
Unitary match: False


In [10]:
# Create input states
test_states = [
    Statevector.from_label('00'),
    Statevector.from_label('01'),
    Statevector.from_label('10'),
    Statevector.from_label('11'),
    Statevector.from_instruction(qc_0),  # Bell state itself
]

# Compare evolutions under U_0 vs U_exp
results = []
for i, sv in enumerate(test_states):
    evolved_true = sv.evolve(U_0)
    evolved_exp = sv.evolve(U_exp)
    fidelity = abs(evolved_true.inner(evolved_exp))  # Magnitude of inner product
    results.append((f'ψ_{i}', fidelity))

for label, overlap in results:
    print(f"{label}: overlap = {overlap:.6f}")


ψ_0: overlap = 0.739866
ψ_1: overlap = 0.336186
ψ_2: overlap = 0.514800
ψ_3: overlap = 0.628811
ψ_4: overlap = 0.663371
